In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import tensorflow_hub as hub
import tensorflow as tf
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn import model_selection, svm
import numpy as np
import spacy
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score


**# ELMO EMBEDDING**

In [0]:
elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

In [0]:
def elmo_vectors(x):
  embeddings = elmo(x.tolist(), signature="default", as_dict=True)["elmo"]

  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    # return average of ELMo features
    return sess.run(tf.reduce_mean(embeddings,1))

In [0]:
df = pd.read_csv("/content/gdrive/My Drive/dataset/data.txt", delimiter=',',encoding = "utf-8")

In [0]:
list_train = [df[i:i+100] for i in range(0,df.shape[0],100)]
elmo_train = [elmo_vectors(x['text']) for x in list_train]

In [0]:
elmo_train_new = np.concatenate(elmo_train, axis = 0)

In [0]:
xtrain, xvalid, ytrain, yvalid = train_test_split(elmo_train_new, 
                                                  df['label'],  
                                                  random_state=42, 
                                                  test_size=0.2)

# SVM

In [0]:
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')

In [37]:
SVM.fit(xtrain,ytrain)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [38]:
predictions_SVM = SVM.predict(xvalid)
print("Accuracy -> ",accuracy_score( yvalid,predictions_SVM)*100)
print("F1 Score -> ",f1_score(yvalid,predictions_SVM, average="macro"))
print("Precision -> ",precision_score(yvalid, predictions_SVM, average="macro"))
print("Recall -> ",recall_score(yvalid,predictions_SVM, average="macro"))

Accuracy ->  92.08333333333333
F1 Score ->  0.8666315697113275
Precision ->  0.9140841813135985
Recall ->  0.834685693165839


# Random Forest

In [43]:
RFC = RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0)
RFC.fit(xtrain,ytrain)
predictions_RFC = RFC.predict(xvalid)
print("Accuracy -> ",accuracy_score( yvalid,predictions_RFC)*100)
print("F1 Score -> ",f1_score(yvalid,predictions_RFC, average="macro"))
print("Precision -> ",precision_score(yvalid, predictions_RFC, average="macro"))
print("Recall -> ",recall_score(yvalid,predictions_RFC, average="macro"))

Accuracy ->  85.83333333333333
F1 Score ->  0.7196508278546141
Precision ->  0.8560280527081459
Recall ->  0.6798645657650462


# Logistic Regression

In [45]:
LR = LogisticRegression(random_state=0)
LR.fit(xtrain,ytrain)
predictions_LR = LR.predict(xvalid)
print("Accuracy -> ",accuracy_score( yvalid,predictions_LR)*100)
print("F1 Score -> ",f1_score(yvalid,predictions_LR, average="macro"))
print("Precision -> ",precision_score(yvalid, predictions_LR, average="macro"))
print("Recall -> ",recall_score(yvalid,predictions_LR, average="macro"))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Accuracy ->  89.33333333333333
F1 Score ->  0.8087639878684656
Precision ->  0.8893039049235993
Recall ->  0.7678840737326407


**# Glove Embedding**

In [12]:
pip install spacy
!python -m spacy download en_core_web_md

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [0]:

class GloveVectorizer(BaseEstimator, TransformerMixin):
    def __init__(self, model_name="en_core_web_md"):
        self._nlp = spacy.load(model_name)

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return np.concatenate(
            [self._nlp(doc).vector.reshape(1, -1) for doc in X]
        )

In [0]:
xtrain, xvalid, ytrain, yvalid = train_test_split(df['text'], df['label'], random_state=42, test_size=0.2)

In [0]:
xtrain,xvalid,ytrain,yvalid = list(xtrain),list(xvalid),list(ytrain),list(yvalid)

# SVM

In [22]:

g = GloveVectorizer()
svm_ = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
pipeline = make_pipeline(g, svm_)

pipeline.fit(xtrain, ytrain)

Pipeline(memory=None,
         steps=[('glovevectorizer', GloveVectorizer(model_name=None)),
                ('svc',
                 SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
                     decision_function_shape='ovr', degree=3, gamma='auto',
                     kernel='linear', max_iter=-1, probability=False,
                     random_state=None, shrinking=True, tol=0.001,
                     verbose=False))],
         verbose=False)

In [23]:
predictions_SVM = pipeline.predict(xvalid)
print("Accuracy -> ",accuracy_score( yvalid,predictions_SVM)*100)
print("F1 Score -> ",f1_score(yvalid,predictions_SVM, average="macro"))
print("Precision -> ",precision_score(yvalid, predictions_SVM, average="macro"))
print("Recall -> ",recall_score(yvalid,predictions_SVM, average="macro"))

Accuracy ->  86.08333333333333
F1 Score ->  0.7712326155623466
Precision ->  0.7959036885652493
Recall ->  0.7534209039428015


# Random Forest

In [25]:
g = GloveVectorizer()
RFC = RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0)
pipeline = make_pipeline(g, RFC)
pipeline.fit(xtrain, ytrain)

Pipeline(memory=None,
         steps=[('glovevectorizer', GloveVectorizer(model_name=None)),
                ('randomforestclassifier',
                 RandomForestClassifier(bootstrap=True, class_weight=None,
                                        criterion='gini', max_depth=3,
                                        max_features='auto',
                                        max_leaf_nodes=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=200, n_jobs=None,
                                        oob_score=False, random_state=0,
                                        verbose=0, warm_start=False))],
         verbose=False)

In [26]:
predictions_RFC = pipeline.predict(xvalid)
print("Accuracy -> ",accuracy_score( yvalid,predictions_RFC)*100)
print("F1 Score -> ",f1_score(yvalid,predictions_RFC, average="macro"))
print("Precision -> ",precision_score(yvalid, predictions_RFC, average="macro"))
print("Recall -> ",recall_score(yvalid,predictions_RFC, average="macro"))

Accuracy ->  82.33333333333334
F1 Score ->  0.5817550505050505
Precision ->  0.8732506997201119
Recall ->  0.5753491000080717


# Logistic Regression

In [27]:
g = GloveVectorizer()
LR = LogisticRegression(random_state=0)
pipeline = make_pipeline(g, LR)
pipeline.fit(xtrain, ytrain)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Pipeline(memory=None,
         steps=[('glovevectorizer', GloveVectorizer(model_name=None)),
                ('logisticregression',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=100,
                                    multi_class='warn', n_jobs=None,
                                    penalty='l2', random_state=0, solver='warn',
                                    tol=0.0001, verbose=0, warm_start=False))],
         verbose=False)

In [28]:
predictions_LR = pipeline.predict(xvalid)
print("Accuracy -> ",accuracy_score( yvalid,predictions_LR)*100)
print("F1 Score -> ",f1_score(yvalid,predictions_LR, average="macro"))
print("Precision -> ",precision_score(yvalid, predictions_LR, average="macro"))
print("Recall -> ",recall_score(yvalid,predictions_LR, average="macro"))

Accuracy ->  85.41666666666666
F1 Score ->  0.7488852297879516
Precision ->  0.7922141335227273
Recall ->  0.7237298792222302
